# RAG over Raku docs

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
September 2024

-------

## Setup

In [ ]:
use LLM::Configurations;
use LLM::RetrievalAugmentedGeneration;
use LLM::RetrievalAugmentedGeneration::VectorDatabase;

use Data::Importers;
use Data::Reshapers;
use Data::Summarizers;

use XDG::BaseDirectory :terms;

use JSON::Fast;
use CBOR::Simple;

use NativeCall;

------

## Ingestion

In [ ]:
my $url = "https://raw.githubusercontent.com/ab5tract/comma-plugin/refs/heads/main/src/main/resources/docs/core.json";

In [ ]:
my $rakuDocs = data-import($url)

In [ ]:
$rakuDocs.keys

In [ ]:
#% html
llm-synthesize([
    "Convert the following Raku code to HTML:", 
    $rakuDocs<ops>.pairs.head(3).raku,
    llm-prompt('NothingElse')('HTML')
    ],
    e => $conf4o
)

In [ ]:
#% html
$rakuDocs<ops>.values[35]

In [ ]:
$rakuDocs<ops>.map({ $_.key => llama-tokenize($_.value.Str) })

In [ ]:
my @tokens = llama-tokenize( $rakuDocs<ops>.map({ $_.value }), format => 'values' )

In [ ]:
sink records-summary(@tokens)

In [ ]:
#% html
my @dsStats = $rakuDocs<ops>.map({  [heading => $_.key, |$_.value.&text-stats] })».Hash;

@dsStats.sort(-*<chars>) ==> to-html(field-names => <heading chars words lines>)